# Make Training Set
Generates a dataset of a few hundreded training points

In [1]:
from mcdemo.utils import get_qm9_path
from ase.calculators.psi4 import Psi4
from ase.io.xyz import read_xyz
from ase import Atoms
from io import StringIO
from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json

Configuration

In [2]:
mol_id = 128
n_samples = 256
method='hf'
basis='3-21g'
element_list = ['C', 'H', 'O', 'N', 'F']

## Get a Molecule
Get a molecule from the QM9 dataset to use to train some models

In [3]:
qm9_path = get_qm9_path()
with gzip.open(qm9_path, 'rt') as fp:
    for _, d in zip(range(mol_id), fp):
        pass
    mol_info = json.loads(d)

In [4]:
print(f'Loaded data for {mol_info["smiles_0"]}')

Loaded data for COCC=O


In [5]:
atoms = next(read_xyz(StringIO(mol_info['xyz'])))

## Compute the Per-Atom Energies
Compute the isolated atom energies with the same functional

In [6]:
isolated_energies = {}
for elem in tqdm(element_list):
    isolated = Atoms(symbols=[elem], positions=[[0, 0, 0]])
    
    # Make the appropriate calculator
    open_shell = sum(isolated.get_atomic_numbers()) % 2 == 1
    if open_shell:
        calc = Psi4(memory='500MB', method=method, basis=basis, reference='uhf', multiplicity=2)
    else:
        calc = Psi4(memory='500MB', method=method, basis=basis)
    isolated_energies[elem] = calc.get_potential_energy(isolated)

100%|██████████| 5/5 [00:01<00:00,  3.18it/s]


In [7]:
with open('isolated-energies.json', 'w') as fp:
    json.dump(isolated_energies, fp, indent=2)

## Generate Many Configurations and Evaluate Their Energy
Get a training set for a machine learning model

In [8]:
calc = Psi4(memory='500MB', method=method, basis=basis, num_threads='max')

In [9]:
rng = np.random.RandomState(1)
results = []
for _ in tqdm(range(n_samples)):
    # Perturb atoms and evaluate energy
    new_atoms = atoms.copy()
    new_atoms.rattle(0.03, rng=rng)
    forces = calc.get_forces(new_atoms)
    energy = calc.get_potential_energy(new_atoms)
    results.append({
        'atoms': new_atoms,
        'total_energy': energy,
        'energy': energy - sum(isolated_energies[e] for e in atoms.symbols),
        'forces': forces
    })

100%|██████████| 256/256 [10:36<00:00,  2.49s/it]


Save as a pickle file

In [10]:
results = pd.DataFrame(results)

In [11]:
results.to_pickle('atoms.pkl.gz')